# Handling libraries

In [1]:
%%capture
# Installing libraries without displaying output
!pip install scikit-learn
!pip install missingno
!pip install xgboost
!pip install imbalanced-learn
!pip install fancyimpute
!pip install tensorflow
!pip install tabulate
!pip install statsmodels
!pip install lightgbm
#!pip install yellowbrick
!pip install scikit-optimize

In [2]:
# Importing necessary libraries
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import RobustScaler
from imblearn.combine import SMOTEENN
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import numpy as np
from sklearn.feature_selection import RFECV
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from imblearn.combine import SMOTEENN
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from imblearn.over_sampling import SMOTENC
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import MinMaxScaler
#from yellowbrick.features import RFECV
from sklearn.neural_network import MLPClassifier


from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.over_sampling import RandomOverSampler
import numpy as np

import lightgbm as lgb

# Data preparation

## Loading data

In [3]:
# Anxiety ------------------------------------
# loading feature engineered variables
anx_data=pd.read_csv('df_anx_t6_2.csv')

seed = 42 

# Loading in data
X_anx = anx_data.iloc[:, :-1]
y_anx = anx_data.iloc[:, -1]


# Distressed ---------------------------------
# loading feature engineered variables
dis_data=pd.read_csv('data_2.csv')

seed = 42 

# Loading in data
X_dis = dis_data.iloc[:, :-1]
y_dis = dis_data.iloc[:, -1]

# Removing redundant features
X_dis = X_dis.drop(X_dis.columns[0], axis=1)


# Bayesian optimization for selected models

## Anxiety

### RF

In [82]:
from skopt import BayesSearchCV
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

# Selected features for anxiety is used
selected_features = ['ucla_t6_sum', 'summary_score_kccq_base', 'eq5d5l_index_t6', 'MCS_t6', 'ImplWeight', 'personality_type_D']
X_selected = X_anx[selected_features]

# Defining search spaces for RF-classifier
rf_search_space = {
    "Classifier__n_estimators": (5, 150),
    "Classifier__max_depth": (None, 1, 50),
    "Classifier__min_samples_split": (2, 10),
    "Classifier__min_samples_leaf": (1, 10),
    'Classifier__max_features': ['log2', 'sqrt'],
    'Classifier__class_weight': ['balanced', 'balanced_subsample', None]
    }

# Defining Classifier
classifier = RandomForestClassifier(random_state=42)
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


# Defining preprocessing steps
scaler = RobustScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),
    ('Classifier', classifier)
])

# Performing Bayesian optimization with F1-macro score
opt = BayesSearchCV(
    pipeline,
    rf_search_space,
    scoring='f1_macro',
    n_iter=30,  # Adjust this based on computational resources and time
    cv=stratified_kfold,
    random_state=42,
    n_jobs=-1,
    verbose=1
)
opt.fit(X_selected, y_anx)

print("Best hyperparameters found:")
print(opt.best_params_)
print(f"Best {classifier.__class__.__name__} cross-validation F1-macro score: {opt.best_score_}\n")


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

### SVM

In [84]:
from skopt import BayesSearchCV
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

selected_features = ['ucla_t6_sum', 'summary_score_kccq_base', 'eq5d5l_index_t6', 'MCS_t6', 'ImplWeight', 'personality_type_D']
X_selected = X_anx[selected_features]


# Defining search spaces for each model
params_svc = {
    'classifier__C': (90, 100),
    'classifier__kernel': ['linear', 'rbf', 'sigmoid'],
    'classifier__degree': (2, 100),
    'classifier__gamma': ['auto'],
    'classifier__coef0': (0, 1),
    'classifier__shrinking': [True, False],
    'classifier__probability': [True, False],
    'classifier__class_weight': ['balanced', None],
    'classifier__max_iter': (4000, 5000)
}

classifier = SVC(random_state=42)
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


# Defining preprocessing steps
scaler = RobustScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),
    ('classifier', classifier)
])

# Performing Bayesian optimization with F1-macro score
opt = BayesSearchCV(
    pipeline,
    params_svc,
    scoring='f1_macro',
    n_iter=30,
    cv=stratified_kfold,
    random_state=42,
    n_jobs=-1,
    verbose=1
)
opt.fit(X_selected, y_anx)

print("Best hyperparameters found:")
print(opt.best_params_)
print(f"Best {classifier.__class__.__name__} cross-validation F1-macro score: {opt.best_score_}\n")

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


### KNN

In [87]:
from skopt import BayesSearchCV
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

# Selected features Anxiety present detection
selected_features = ['ucla_t6_sum', 'summary_score_kccq_base', 'eq5d5l_index_t6', 'MCS_t6', 'ImplWeight', 'personality_type_D']
X_selected = X_anx[selected_features]


# Defining search spaces for each model
params_knn = {
    'classifier__n_neighbors': (1, 10),
    'classifier__weights': ['uniform', 'distance'],
    'classifier__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'classifier__leaf_size': (1, 20),
    'classifier__p': [1, 2],
}

classifier = KNeighborsClassifier(n_jobs=-1)
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


# Defining preprocessing steps
scaler = RobustScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),
    ('classifier', classifier)
])

# Performing Bayesian optimization with F1-macro score
opt = BayesSearchCV(
    pipeline,
    params_knn,
    scoring='f1_macro',
    n_iter=30, 
    cv=stratified_kfold,
    random_state=42,
    n_jobs=-1,
    verbose=1
)
opt.fit(X_selected, y_anx)

print("Best hyperparameters found:")
print(opt.best_params_)
print(f"Best {classifier.__class__.__name__} cross-validation F1-macro score: {opt.best_score_}\n")

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/opt/conda/lib/python3.10/site-packages/skopt/space/space.py:110: UserWarning: Dimension [1, 2] was inferred to Integer(low=1, high=2, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(1, 2), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/skopt/space/space.py:110: UserWarning: Dimension [1, 2] was inferred to Integer(low=1, high=2, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(1, 2), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/skopt/space/space.py:110: UserWarning: Dimension [1, 2] was inferred to Integer(low=1, high=2, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categoric

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

### XGB

In [25]:
selected_features = ['ucla_t6_sum', 'summary_score_kccq_base', 'eq5d5l_index_t6', 'MCS_t6', 'ImplWeight', 'personality_type_D']
X_selected = X_anx[selected_features]


# Defining search spaces for each model
params_xgb = {
    'classifier__n_estimators': (5, 200),
    'classifier__max_depth': (1, 50),
    'classifier__learning_rate': (0.0001, 1),
    'classifier__min_child_weight': (1, 15),
    'classifier__subsample': [0.3, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
    'classifier__colsample_bytree': [0.3, 0.5, 0.7, 0.75, 0.8, 0.85, 0.9, 1],
    'classifier__n_jobs': [-1]
}

# Defining classifier
classifier = xgb.XGBClassifier(random_state=42, n_jobs=-1)
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


# Defining preprocessing steps
scaler = RobustScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),
    ('classifier', classifier)
])

# Performing Bayesian optimization with F1-macro score
opt = BayesSearchCV(
    pipeline,
    params_xgb,
    scoring='f1_macro',
    n_iter=50, 
    cv=stratified_kfold,
    random_state=42,
    n_jobs=-1,
    verbose=1
)
opt.fit(X_selected, y_anx)

print("Best hyperparameters found:")
print(opt.best_params_)
print(f"Best {classifier.__class__.__name__} cross-validation F1-macro score: {opt.best_score_}\n")

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

### LGBM

In [91]:
selected_features = ['ucla_t6_sum', 'summary_score_kccq_base', 'eq5d5l_index_t6', 'MCS_t6', 'ImplWeight', 'personality_type_D']
X_selected = X_anx[selected_features]


# Defining search spaces for each model
params_lgbm = {
    'classifier__n_estimators': (5, 300),
    'classifier__boosting_type': ['gbdt', 'dart'],
    'classifier__num_leaves': (2, 50),
    'classifier__max_depth': (-1, 50),  #Default = -1
    'classifier__learning_rate': [0.001, 0.05, 0.1, 0.2, 0.3, 0.5, 1],  #Default = 0.1
    'classifier__objective': ['binary', None], #Default = None. 
    }

# Defining classifier
classifier = lgb.LGBMClassifier(random_state=42, n_jobs =-1)
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


# Defining preprocessing steps
scaler = RobustScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),
    ('classifier', classifier)
])

# Performing Bayesian optimization with F1-macro score
opt = BayesSearchCV(
    pipeline,
    params_lgbm,
    scoring='f1_macro',
    n_iter=30,
    cv=stratified_kfold,
    random_state=42,
    n_jobs=-1,
    verbose=1
)
opt.fit(X_selected, y_anx)

print("Best hyperparameters found:")
print(opt.best_params_)
print(f"Best {classifier.__class__.__name__} cross-validation F1-macro score: {opt.best_score_}\n")

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

## Distress

### RF

In [76]:
from skopt import BayesSearchCV
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

# Best selected features for Distressed is used
selected_features = ['ucla_t6_sum', 'eq5d5l_index_t6', 'depression_base_score', 'icdc_t6_sum', 'MCS_t6', 'Return_to_function_t6', 'age', 'anxiety_base_score']
X_dis = X_dis[selected_features]
X_selected = X_dis[selected_features]

# Defining search spaces for each model
rf_search_space = {
    "Classifier__n_estimators": (30, 130),
    "Classifier__max_depth": (5, 100),
    "Classifier__min_samples_split": (2, 100),
    "Classifier__min_samples_leaf": (1, 50),
    'Classifier__max_features': ['log2', 'sqrt'],
    'Classifier__class_weight': ['balanced', 'balanced_subsample', None]
    }

# Defining classifier
classifier = RandomForestClassifier(random_state=42)
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


# Defining preprocessing steps
scaler = MinMaxScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),
    ('Classifier', classifier)
])

# Performing Bayesian optimization with F1-macro score
opt = BayesSearchCV(
    pipeline,
    rf_search_space,
    scoring='f1_macro',
    n_iter=30, 
    cv=stratified_kfold,
    random_state=42,
    n_jobs=-1,
    verbose=1
)
opt.fit(X_selected, y_dis)

print("Best hyperparameters found:")
print(opt.best_params_)
print(f"Best {classifier.__class__.__name__} cross-validation F1-macro score: {opt.best_score_}\n")


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

/opt/conda/lib/python3.10/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point ['balanced_subsample', 5, 'sqrt', 1, 2, 30] before, using random point ['balanced_subsample', 74, 'sqrt', 11, 92, 108]
  warnings.warn(


Fitting 5 folds for each of 1 candidates, totalling 5 fits


/opt/conda/lib/python3.10/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point ['balanced_subsample', 5, 'sqrt', 1, 2, 30] before, using random point ['balanced', 57, 'log2', 6, 25, 45]
  warnings.warn(


Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/opt/conda/lib/python3.10/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point ['balanced_subsample', 5, 'sqrt', 1, 2, 30] before, using random point ['balanced', 73, 'sqrt', 39, 87, 72]
  warnings.warn(


Fitting 5 folds for each of 1 candidates, totalling 5 fits


/opt/conda/lib/python3.10/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point ['balanced_subsample', 5, 'sqrt', 1, 2, 30] before, using random point ['balanced_subsample', 75, 'log2', 15, 91, 49]
  warnings.warn(


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best hyperparameters found:
OrderedDict([('Classifier__class_weight', 'balanced'), ('Classifier__max_depth', 5), ('Classifier__max_features', 'sqrt'), ('Classifier__min_samples_leaf', 1), ('Classifier__min_samples_split', 2), ('Classifier__n_estimators', 30)])
Best RandomForestClassifier cross-validation F1-macro score: 0.9088854087089381



### SVM

In [70]:
from skopt import BayesSearchCV
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

selected_features = ['ucla_t6_sum', 'eq5d5l_index_t6', 'depression_base_score', 'icdc_t6_sum', 'MCS_t6', 'Return_to_function_t6', 'age', 'anxiety_base_score']
X_selected = X_dis[selected_features]


# Defining search spaces for each model
params_svc = {
    'classifier__C': (90, 100),
    'classifier__kernel': ['linear', 'rbf', 'sigmoid'],
    'classifier__degree': (2, 100),
    'classifier__gamma': ['auto'],
    'classifier__coef0': (0, 1),
    'classifier__shrinking': [True, False],
    'classifier__probability': [True, False],
    'classifier__class_weight': ['balanced', None],
    'classifier__max_iter': (4000, 5000)
}

# Defining classifier
classifier = SVC(random_state=42)
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


# Defining preprocessing steps
scaler = MinMaxScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),
    ('classifier', classifier)
])

# Performing Bayesian optimization with F1-macro score
opt = BayesSearchCV(
    pipeline,
    params_svc,
    scoring='f1_macro',
    n_iter=30,
    cv=stratified_kfold,
    random_state=42,
    n_jobs=-1,
    verbose=1
)
opt.fit(X_selected, y_dis)

print("Best hyperparameters found:")
print(opt.best_params_)
print(f"Best {classifier.__class__.__name__} cross-validation F1-macro score: {opt.best_score_}\n")

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

### KNN

In [58]:
from skopt import BayesSearchCV
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier

selected_features = ['ucla_t6_sum', 'eq5d5l_index_t6', 'depression_base_score', 'icdc_t6_sum', 'MCS_t6', 'Return_to_function_t6', 'age', 'anxiety_base_score']
X_selected = X_dis[selected_features]


# Defining search spaces for each model
params_knn = {
    'classifier__n_neighbors': (1, 10),
    'classifier__weights': ['uniform', 'distance'],
    'classifier__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'classifier__leaf_size': (1, 20),
    'classifier__p': [1, 2],
}

# Defining classifier
classifier = KNeighborsClassifier(n_jobs=-1)
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


# Defining preprocessing steps
scaler = MinMaxScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),
    ('classifier', classifier)
])

# Performing Bayesian optimization with F1-macro score
opt = BayesSearchCV(
    pipeline,
    params_knn,
    scoring='f1_macro',
    n_iter=30, 
    cv=stratified_kfold,
    random_state=42,
    n_jobs=-1,
    verbose=1
)
opt.fit(X_selected, y_dis)

print("Best hyperparameters found:")
print(opt.best_params_)
print(f"Best {classifier.__class__.__name__} cross-validation F1-macro score: {opt.best_score_}\n")

/opt/conda/lib/python3.10/site-packages/skopt/space/space.py:110: UserWarning: Dimension [1, 2] was inferred to Integer(low=1, high=2, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(1, 2), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/skopt/space/space.py:110: UserWarning: Dimension [1, 2] was inferred to Integer(low=1, high=2, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(1, 2), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/skopt/space/space.py:110: UserWarning: Dimension [1, 2] was inferred to Integer(low=1, high=2, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categoric

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

/opt/conda/lib/python3.10/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point ['ball_tree', 3, 6, 2, 'distance'] before, using random point ['kd_tree', 7, 3, 1, 'distance']
  warnings.warn(


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best hyperparameters found:
OrderedDict([('classifier__algorithm', 'brute'), ('classifier__leaf_size', 4), ('classifier__n_neighbors', 6), ('classifier__p', 2), ('classifier__weights', 'distance')])
Best KNeighborsClassifier cross-validation F1-macro score: 0.8929530181562623



### XGB

In [61]:
selected_features = ['ucla_t6_sum', 'eq5d5l_index_t6', 'depression_base_score', 'icdc_t6_sum', 'MCS_t6', 'Return_to_function_t6', 'age', 'anxiety_base_score']
X_selected = X_dis[selected_features]


# Defining search spaces for each model
params_xgb = {
    'classifier__n_estimators': (5, 100),
    'classifier__max_depth': (1, 50),
    'classifier__learning_rate': (0.0001, 1),
    'classifier__min_child_weight': (1, 15),
    'classifier__subsample': [0.3, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
    'classifier__colsample_bytree': [0.3, 0.5, 0.7, 0.75, 0.8, 0.85, 0.9, 1],
    'classifier__n_jobs': [-1]
}

# Defining classifier
classifier = xgb.XGBClassifier(random_state=42)
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


# Defining preprocessing steps
scaler = MinMaxScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),
    ('classifier', classifier)
])

# Performing Bayesian optimization with F1-macro score
opt = BayesSearchCV(
    pipeline,
    params_xgb,
    scoring='f1_macro',
    n_iter=30,
    cv=stratified_kfold,
    random_state=42,
    n_jobs=-1,
    verbose=1
)
opt.fit(X_selected, y_dis)

print("Best hyperparameters found:")
print(opt.best_params_)
print(f"Best {classifier.__class__.__name__} cross-validation F1-macro score: {opt.best_score_}\n")

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

### LGBM

In [65]:
selected_features = ['ucla_t6_sum', 'eq5d5l_index_t6', 'depression_base_score', 'icdc_t6_sum', 'MCS_t6', 'Return_to_function_t6', 'age', 'anxiety_base_score']
X_selected = X_dis[selected_features]


# Defining search spaces for each model
params_lgbm = {
    'classifier__n_estimators': (5, 150),
    'classifier__boosting_type': ['gbdt', 'dart'],
    'classifier__num_leaves': (2, 25),
    'classifier__max_depth': (-1, 25),  #Default = -1
    'classifier__learning_rate': [0.001, 0.05, 0.1, 0.2, 0.3, 0.5, 1],  #Default = 0.1
    'classifier__objective': ['binary', None], #Default = None. 
    'classifier__class_weight': [None, 'balanced'], #Default = None
    'classifier__colsample_bytree': [0.3, 0.5, 0.7, 0.75, 0.8, 0.85, 0.9, 1] #Default = 1
    }

# Defining classifier
classifier = lgb.LGBMClassifier(random_state=42, n_jobs =-1)
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


# Defining preprocessing steps
scaler = MinMaxScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),
    ('classifier', classifier)
])

# Performing Bayesian optimization with F1-macro score
opt = BayesSearchCV(
    pipeline,
    params_lgbm,
    scoring='f1_macro',
    n_iter=30, 
    cv=stratified_kfold,
    random_state=42,
    n_jobs=-1,
    verbose=1
)
opt.fit(X_selected, y_dis)

print("Best hyperparameters found:")
print(opt.best_params_)
print(f"Best {classifier.__class__.__name__} cross-validation F1-macro score: {opt.best_score_}\n")

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

# Random search results (f1-macro + std)

In [1]:
# Defining variables for validation segments

# Anxiety present detection
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
selected_features = ['ucla_t6_sum', 'summary_score_kccq_base', 'eq5d5l_index_t6', 'MCS_t6', 'ImplWeight', 'personality_type_D']
X_anx = X_anx[selected_features]

# Distress present detection
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
selected_features = ['ucla_t6_sum', 'eq5d5l_index_t6', 'depression_base_score', 'icdc_t6_sum', 'MCS_t6', 'Return_to_function_t6', 'age', 'anxiety_base_score']
X_dis = X_dis[selected_features]

NameError: name 'StratifiedKFold' is not defined

## Anxiety

### RF

#### Performing RandomSearch

In [38]:
import lightgbm as lgb
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
import numpy as np

# Defining type of split for CV - this is used for imbalanced datasets to keep equal proportion of class balance between folds
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Defining model and resampling method
model = RandomForestClassifier(random_state=42)

# Defining parameter grid
params_rf = {}
params_rf['classifier__n_estimators'] = [5, 20, 35, 50, 75, 100, 150, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
params_rf['classifier__max_depth'] = [None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
params_rf['classifier__min_samples_split'] = [2, 3, 5, 6, 7, 8, 9, 10]
params_rf['classifier__min_samples_leaf'] = [1, 2, 3, 5, 6, 7, 8, 9, 10]
params_rf['classifier__max_features'] = ['log2', 'sqrt']
params_rf['classifier__class_weight'] = ['balanced', 'balanced_subsample', None]

# Defining preprocessing methods for pipeline
scaler = RobustScaler()
imputer = SimpleImputer()


# Defining pipeline to avoid data leakage
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),    
    ('classifier', model),  
    ])


############################################################################################

# Initializing RandomizedSearchCV
opt = RandomizedSearchCV(
    pipeline,
    param_distributions=params_rf,
    n_iter=10000,  # Number of parameter settings that are sampled
    cv=stratified_kfold,  # Number of cross-validation folds
    random_state=42,
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1
)

opt.fit(X_anx_selected, y_anx)

#printing(opt.best_score_)
print(opt.best_score_)
print(opt.best_params_)

Fitting 5 folds for each of 10000 candidates, totalling 50000 fits
0.9209040685715605
{'classifier__n_estimators': 100, 'classifier__min_samples_split': 7, 'classifier__min_samples_leaf': 3, 'classifier__max_features': 'sqrt', 'classifier__max_depth': 7, 'classifier__class_weight': 'balanced'}


#### Validating results

In [18]:
# Defining classifier with best params
classifier = RandomForestClassifier(n_estimators = 100,
min_samples_split = 7,
min_samples_leaf = 3,
max_features = 'sqrt',
max_depth = 7,
class_weight = 'balanced',
random_state=42)

# Defining preprocessing steps
scaler = RobustScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),
    ('Classifier', classifier)
])

# Performing cross-validation
scores = cross_val_score(pipeline, X_anx, y_anx, cv=stratified_kfold, scoring='f1_macro', n_jobs=-1)
mean_score = np.mean(scores)
std_score = np.std(scores)

# Printing results
print()
print(classifier.__class__.__name__)
print("Mean F1-macro Score:", mean_score, "std:", std_score)


RandomForestClassifier
Mean F1-macro Score: 0.9209040685715605 std: 0.05209723108198606


### SVM

#### Performing RandomSearch

In [39]:
import lightgbm as lgb
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
import numpy as np
from sklearn.svm import SVC

# Defining type of split for CV - this is used for imbalanced datasets to keep equal proportion of class balance between folds
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Defining model and resampling method
model = SVC(random_state=42)

# Defining parameter grid
params_svc = {
    'classifier__C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0],
    'classifier__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'classifier__degree': [2, 3, 4, 6, 10, 15, 20, 40, 100, 150, 200, 400, 700, 1000],
    'classifier__gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1, 2, 5, 10],
    'classifier__coef0': [0.0, 0.1, 0.5, 1.0],
    'classifier__shrinking': [True, False],
    'classifier__probability': [True, False],
    'classifier__class_weight': ['balanced', None],
    'classifier__max_iter': [500, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
}

# Defining preprocessing methods for pipeline
scaler = RobustScaler()
imputer = SimpleImputer()

pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),    
    ('classifier', model),  
    ])

############################################################################################

# Initializing RandomizedSearchCV
opt = RandomizedSearchCV(
    pipeline,
    param_distributions=params_svc,
    n_iter=40000,  # Number of parameter settings that are sampled
    cv=stratified_kfold,  # Number of cross-validation folds
    random_state=42,
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1
)

opt.fit(X_anx_selected, y_anx)

#printing(opt.best_score_)
print(opt.best_score_)
print(opt.best_params_)

Fitting 5 folds for each of 40000 candidates, totalling 200000 fits


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
19190 fits failed out of a total of 200000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
19190 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.10/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/imblearn/pipeline.py", line 326, in fit
    self._final_estimator.fit(Xt, yt, **last_step_params["fit"])
  File "/opt/co

0.9126825719632198
{'classifier__shrinking': True, 'classifier__probability': False, 'classifier__max_iter': 8000, 'classifier__kernel': 'rbf', 'classifier__gamma': 'auto', 'classifier__degree': 15, 'classifier__coef0': 0.1, 'classifier__class_weight': None, 'classifier__C': 1.0}


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_search.py:1051: UserWarning: One or more of the test scores are non-finite: [0.47995118 0.48099838 0.65547234 ... 0.90639697 0.31609963 0.69100935]
  warnings.warn(


#### Validating results

In [19]:
# Defining classifier with best params
classifier = SVC(shrinking = True,
probability = False,
max_iter = 8000,
kernel = 'rbf',
gamma = 'auto',
degree = 15,
coef0 = 0.1,
class_weight = None,
C = 1.0,
random_state=42)

# Defining preprocessing steps
scaler = RobustScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),
    ('Classifier', classifier)
])

# Performing cross-validation
scores = cross_val_score(pipeline, X_anx, y_anx, cv=stratified_kfold, scoring='f1_macro', n_jobs=-1)
mean_score = np.mean(scores)
std_score = np.std(scores)

# Printing results
print()
print(classifier.__class__.__name__)
print("Mean F1-macro Score:", mean_score, "std:", std_score)


SVC
Mean F1-macro Score: 0.9126825719632198 std: 0.08203534839628797


### KNN

#### Performing RandomSearch

In [40]:
import lightgbm as lgb
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import make_scorer, f1_score
import numpy as np

# Defining type of split for CV - this is used for imbalanced datasets to keep equal proportion of class balance between folds
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Defining model and resampling method
model = KNeighborsClassifier()

params_knn = {} 
params_knn['classifier__n_neighbors'] = list(range(1, 41, 2)) #Default = 5
params_knn['classifier__weights'] = ['uniform', 'distance'] #Default = uniform
params_knn['classifier__algorithm'] = ['auto', 'ball_tree', 'kd_tree', 'brute'] #Default = Auto
params_knn['classifier__leaf_size'] = [10, 20, 30, 40, 50] # Default = 30. Only for Balltree and KDTree
params_knn['classifier__p'] = [1, 2] #Default = 2

# Defining preprocessing methods for pipeline
scaler = RobustScaler()
imputer = SimpleImputer()

# Defining pipeline to avoid data leakage
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),  
    ('classifier', model),  
    ])

############################################################################################

# Initialize RandomizedSearchCV
opt = RandomizedSearchCV(
    pipeline,
    param_distributions=params_knn,
    n_iter=10000,  # Number of parameter settings that are sampled
    cv=stratified_kfold,  # Number of cross-validation folds
    random_state=42,
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1
)

opt.fit(X_anx_selected, y_anx)

#printing(opt.best_score_)
print(opt.best_score_)
print(opt.best_params_)

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_search.py:318: UserWarning: The total space of parameters 1600 is smaller than n_iter=10000. Running 1600 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 1600 candidates, totalling 8000 fits
0.912602819606272
{'classifier__weights': 'uniform', 'classifier__p': 1, 'classifier__n_neighbors': 5, 'classifier__leaf_size': 10, 'classifier__algorithm': 'auto'}


#### Validating results

In [20]:
# Defining classifier with best params
classifier = KNeighborsClassifier(weights = 'uniform',
p = 1,
n_neighbors = 5,
leaf_size = 10,
algorithm = 'auto',
n_jobs=-1)

# Defining preprocessing steps
scaler = RobustScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),
    ('Classifier', classifier)
])

# Performing cross-validation
scores = cross_val_score(pipeline, X_anx, y_anx, cv=stratified_kfold, scoring='f1_macro', n_jobs=-1)
mean_score = np.mean(scores)
std_score = np.std(scores)

# Printing results
print()
print(classifier.__class__.__name__)
print("Mean F1-macro Score:", mean_score, "std:", std_score)


KNeighborsClassifier
Mean F1-macro Score: 0.912602819606272 std: 0.07830052829708159


### XGB

#### Performing GridSearch

In [41]:
import lightgbm as lgb
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import make_scorer, f1_score
import numpy as np

# Defining type of split for CV - this is used for imbalanced datasets to keep equal proportion of class balance between folds
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Defining model and resampling method
model = xgb.XGBClassifier(random_state=42)

params_xgb = {}
params_xgb['classifier__n_estimators'] =  [5, 20, 35, 50, 75, 100, 150, 200, 300, 400, 500, 600, 700, 800, 900, 1000]  #Default = 100. 75 was best during early testing
params_xgb['classifier__max_depth'] = [None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] #Default = 6
params_xgb['classifier__learning_rate'] = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1] #Default = 0.3
params_xgb['classifier__min_child_weight'] = [1, 2, 3, 4, 5, 6, 7] #Default = 1
params_xgb['classifier__subsample'] = [0.3, 0.5, 0.6, 0.7, 0.8, 0.9, 1] #Default = 1
params_xgb['classifier__colsample_bytree'] = [0.3, 0.5, 0.7, 0.75, 0.8, 0.85, 0.9, 1] #Default = 1
params_xgb['classifier__n_jobs'] = [-1]

# Defining preprocessing methods for pipeline
scaler = RobustScaler()
imputer = SimpleImputer()

# Defininging pipeline to avoid data leakage
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),    
    ('classifier', model),  
    ])

############################################################################################

# Initializing RandomizedSearchCV
opt = RandomizedSearchCV(
    pipeline,
    param_distributions=params_xgb,
    n_iter=10000,  # Number of parameter settings that are sampled
    cv=stratified_kfold,  # Number of cross-validation folds
    random_state=42,
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1
)

opt.fit(X_anx_selected, y_anx)

#printing(opt.best_score_)
print(opt.best_score_)
print(opt.best_params_)

Fitting 5 folds for each of 10000 candidates, totalling 50000 fits
0.9137019004553254
{'classifier__subsample': 1, 'classifier__n_jobs': -1, 'classifier__n_estimators': 150, 'classifier__min_child_weight': 1, 'classifier__max_depth': 3, 'classifier__learning_rate': 0.5, 'classifier__colsample_bytree': 1}


#### Validating results

In [21]:
# Defining classifier with best params
classifier = xgb.XGBClassifier(subsample = 1,
n_estimators = 150,
min_child_weight = 1,
max_depth = 3,
learning_rate = 0.5,
colsample_bytree = 1,
random_state=42, n_jobs=-1)

# Defining preprocessing steps
scaler = RobustScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),
    ('Classifier', classifier)
])

# Performing cross-validation
scores = cross_val_score(pipeline, X_anx, y_anx, cv=stratified_kfold, scoring='f1_macro', n_jobs=-1)
mean_score = np.mean(scores)
std_score = np.std(scores)

# Printing results
print()
print(classifier.__class__.__name__)
print("Mean F1-macro Score:", mean_score, "std:", std_score)


XGBClassifier
Mean F1-macro Score: 0.9137019004553254 std: 0.08263086794122673


### LGBM

#### Performing RandomSearch

In [42]:
import lightgbm as lgb
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import make_scorer, f1_score
import numpy as np

# Defining type of split for CV - this is used for imbalanced datasets to keep equal proportion of class balance between folds
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Defining model and resampling method
model = lgb.LGBMClassifier(random_state=42)

params_lgbm = {}
params_lgbm['classifier__boosting_type'] = ['gbdt', 'dart']  #Default = gbdt
params_lgbm['classifier__n_estimators'] = [5, 10, 15, 20, 25, 35, 50, 75, 100, 150, 200, 300, 400] #Default = 100. Good performance around 20
params_lgbm['classifier__num_leaves'] = [1, 3, 5, 7, 9, 11, 15] # 20, 25, 31, 40, 50] #Default = 31
params_lgbm['classifier__max_depth'] = [-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  #Default = -1
params_lgbm['classifier__learning_rate'] = [0.001, 0.05, 0.1, 0.2, 0.3, 0.5, 1]  #Default = 0.1
params_lgbm['classifier__objective'] = ['binary', None] #Default = None. 
params_lgbm['classifier__class_weight'] = [None, 'balanced'] #Default = None
params_lgbm['classifier__colsample_bytree'] = [0.3, 0.5, 0.7, 0.75, 0.8, 0.85, 0.9, 1] #Default = 1

# Defining preprocessing methods for pipeline
scaler = RobustScaler()
imputer = SimpleImputer()

# Defining pipeline to avoid data leakage
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),   
    ('classifier', model),  
    ])
############################################################################################

# Initializing RandomizedSearchCV
opt = RandomizedSearchCV(
    pipeline,
    param_distributions=params_lgbm,
    n_iter=5000,  # Number of parameter settings that are sampled
    cv=stratified_kfold,  # Number of cross-validation folds
    random_state=42,
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1
)

opt.fit(X_anx_selected, y_anx)

#printing(opt.best_score_)
print(opt.best_score_)
print(opt.best_params_)

Fitting 5 folds for each of 5000 candidates, totalling 25000 fits


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
3615 fits failed out of a total of 25000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3615 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.10/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/imblearn/pipeline.py", line 326, in fit
    self._final_estimator.fit(Xt, yt, **last_step_params["fit"])
  File "/opt/conda

[LightGBM] [Info] Number of positive: 35, number of negative: 443
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000097 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 309
[LightGBM] [Info] Number of data points in the train set: 478, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.073222 -> initscore=-2.538222
[LightGBM] [Info] Start training from score -2.538222
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_search.py:1051: UserWarning: One or more of the test scores are non-finite: [0.7591562  0.48099838 0.87385151 ... 0.86977434 0.87378481 0.85965462]
  warnings.warn(


#### Validating results

In [22]:
# Defining classifier with best params
classifier = lgb.LGBMClassifier(objective = None,
num_leaves = 15,
n_estimators = 20,
max_depth = 6,
learning_rate = 0.1,
colsample_bytree = 0.75,
class_weight = None,
boosting_type = 'gbdt',
random_state=42, n_jobs =-1)

# Defining preprocessing steps
scaler = RobustScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),
    ('Classifier', classifier)
])

# Performing cross-validation
scores = cross_val_score(pipeline, X_anx, y_anx, cv=stratified_kfold, scoring='f1_macro', n_jobs=-1)
mean_score = np.mean(scores)
std_score = np.std(scores)

# Printing results
print()
print(classifier.__class__.__name__)
print("Mean F1-macro Score:", mean_score, "std:", std_score)


LGBMClassifier
Mean F1-macro Score: 0.910054117835578 std: 0.07139540921071352


## Distressed

### RF

#### Performing RandomSearch

In [25]:
import lightgbm as lgb
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
import numpy as np

# Defining type of split for CV - this is used for imbalanced datasets to keep equal proportion of class balance between folds
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Defining model and resampling method
model = RandomForestClassifier(random_state=42)

# Defining parameter grid
params_rf = {}
params_rf['classifier__n_estimators'] = [5, 20, 35, 50, 75, 100, 150, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
params_rf['classifier__max_depth'] = [None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
params_rf['classifier__min_samples_split'] = [2, 3, 5, 6, 7, 8, 9, 10]
params_rf['classifier__min_samples_leaf'] = [1, 2, 3, 5, 6, 7, 8, 9, 10]
params_rf['classifier__max_features'] = ['log2', 'sqrt']
params_rf['classifier__class_weight'] = ['balanced', 'balanced_subsample', None]

# Defining preprocessing methods for pipeline
scaler = MinMaxScaler()
imputer = SimpleImputer()


# Defining pipeline to avoid data leakage
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),    
    ('classifier', model),  
    ])


############################################################################################

# Initializing RandomizedSearchCV
opt = RandomizedSearchCV(
    pipeline,
    param_distributions=params_rf,
    n_iter=10000,  # Number of parameter settings that are sampled
    cv=stratified_kfold,  # Number of cross-validation folds
    random_state=42,
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1
)

opt.fit(X_dis_selected, y_dis)

# printing(opt.best_score_)
print(opt.best_score_)
print(opt.best_params_)

Fitting 5 folds for each of 10000 candidates, totalling 50000 fits
0.9154748466102941
{'classifier__n_estimators': 50, 'classifier__min_samples_split': 3, 'classifier__min_samples_leaf': 1, 'classifier__max_features': 'sqrt', 'classifier__max_depth': 7, 'classifier__class_weight': None}


#### Validating results

In [29]:
# Defining classifier with best params
classifier = RandomForestClassifier(n_estimators = 50,
min_samples_split = 3,
min_samples_leaf = 1,
max_features = 'sqrt',
max_depth = 7,
class_weight = None,
random_state=42)

# Defining preprocessing steps
scaler = MinMaxScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('MinMax', scaler),
    ('Simple', imputer),
    ('Classifier', classifier)
])

# Performing cross-validation
scores = cross_val_score(pipeline, X_dis, y_dis, cv=stratified_kfold, scoring='f1_macro', n_jobs=-1)
mean_score = np.mean(scores)
std_score = np.std(scores)

# Printing results
print()
print(classifier.__class__.__name__)
print("Mean F1-macro Score:", mean_score, "std:", std_score)


RandomForestClassifier
Mean F1-macro Score: 0.9154748466102941 std: 0.08290499798612358


### SVM

#### Performing RandomSearch

In [26]:
import lightgbm as lgb
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
import numpy as np
from sklearn.svm import SVC

# Defining type of split for CV - this is used for imbalanced datasets to keep equal proportion of class balance between folds
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Defining model and resampling method
model = SVC(random_state=42)

# Defining parameter grid
params_svc = {}
params_svc['classifier__C'] = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0],
params_svc['classifier__kernel'] = ['linear', 'poly', 'rbf', 'sigmoid'],
params_svc['classifier__degree'] = [2, 3, 4, 6, 10, 15, 20, 40, 100, 150, 200, 400, 700, 1000],
params_svc['classifier__gamma'] = ['scale', 'auto', 0.001, 0.01, 0.1, 1, 2, 5, 10],
params_svc['classifier__coef0'] = [0.0, 0.1, 0.5, 1.0],
params_svc['classifier__shrinking'] = [True, False],
params_svc['classifier__probability'] = [True, False],
params_svc['classifier__class_weight'] = ['balanced', None],
params_svc['classifier__max_iter'] = [500, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

# Defining preprocessing methods for pipeline
scaler = MinMaxScaler()
imputer = SimpleImputer()

pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),    
    ('classifier', model),  
    ])

############################################################################################

# Initializing RandomizedSearchCV
opt = RandomizedSearchCV(
    pipeline,
    param_distributions=params_svc,
    n_iter=40000,  # Number of parameter settings that are sampled
    cv=stratified_kfold,  # Number of cross-validation folds
    random_state=42,
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1
)

opt.fit(X_dis_selected, y_dis)

#printing(opt.best_score_)
print(opt.best_score_)
print(opt.best_params_)

Fitting 5 folds for each of 40000 candidates, totalling 200000 fits


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
13414 fits failed out of a total of 200000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
13414 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.10/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/imblearn/pipeline.py", line 326, in fit
    self._final_estimator.fit(Xt, yt, **last_step_params["fit"])
  File "/opt/co

0.8889376147546265
{'classifier__shrinking': False, 'classifier__probability': False, 'classifier__max_iter': 6000, 'classifier__kernel': 'sigmoid', 'classifier__gamma': 'auto', 'classifier__degree': 6, 'classifier__coef0': 0.5, 'classifier__class_weight': None, 'classifier__C': 100.0}


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_search.py:1051: UserWarning: One or more of the test scores are non-finite: [0.50148069 0.47873578 0.76638245 ... 0.86357138 0.39866537 0.76118185]
  warnings.warn(


#### Validating results

In [30]:
# Defining classifier with best params
classifier = SVC(shrinking = False,
probability = False,
max_iter = 6000,
kernel = 'sigmoid',
gamma = 'auto',
degree = 6,
coef0 = 0.5,
class_weight = None,
C = 100.0,
random_state=42)

# Defining preprocessing steps
scaler = MinMaxScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('MinMax', scaler),
    ('Simple', imputer),
    ('Classifier', classifier)
])

# Performing cross-validation
scores = cross_val_score(pipeline, X_dis, y_dis, cv=stratified_kfold, scoring='f1_macro', n_jobs=-1)
mean_score = np.mean(scores)
std_score = np.std(scores)

# Printing results
print()
print(classifier.__class__.__name__)
print("Mean F1-macro Score:", mean_score, "std:", std_score)


SVC
Mean F1-macro Score: 0.8889376147546265 std: 0.11453618279448234


### KNN

#### Performing RandomSearch

In [20]:
import lightgbm as lgb
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import make_scorer, f1_score
import numpy as np

# Defining type of split for CV - this is used for imbalanced datasets to keep equal proportion of class balance between folds
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Defining model and resampling method
model = KNeighborsClassifier()

params_knn = {} 
params_knn['classifier__n_neighbors'] = list(range(1, 41, 2)) #Default = 5
params_knn['classifier__weights'] = ['uniform', 'distance'] #Default = uniform
params_knn['classifier__algorithm'] = ['auto', 'ball_tree', 'kd_tree', 'brute'] #Default = Auto
params_knn['classifier__leaf_size'] = [10, 20, 30, 40, 50] # Default = 30. Only for Balltree and KDTree
params_knn['classifier__p'] = [1, 2] #Default = 2

# Defining preprocessing methods for pipeline
scaler = MinMaxScaler()
imputer = SimpleImputer()

# Defining pipeline to avoid data leakage
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),    
    ('classifier', model),  
    ])

############################################################################################

# Initializing RandomizedSearchCV
opt = RandomizedSearchCV(
    pipeline,
    param_distributions=params_knn,
    n_iter=10000,  # Number of parameter settings that are sampled
    cv=stratified_kfold,  # Number of cross-validation folds
    random_state=42,
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1
)

opt.fit(X_dis_selected, y_dis)

#printing(opt.best_score_)
print(opt.best_score_)
print(opt.best_params_)

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_search.py:318: UserWarning: The total space of parameters 1600 is smaller than n_iter=10000. Running 1600 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 1600 candidates, totalling 8000 fits
0.8719658963464401
{'classifier__weights': 'uniform', 'classifier__p': 2, 'classifier__n_neighbors': 5, 'classifier__leaf_size': 10, 'classifier__algorithm': 'auto'}


#### Validating results

In [31]:
# Defining classifier with best params
classifier = KNeighborsClassifier(weights = 'uniform',
p = 2,
n_neighbors = 5,
leaf_size = 10,
algorithm = 'auto',
n_jobs=-1)

# Defining preprocessing steps
scaler = MinMaxScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('MinMax', scaler),
    ('Simple', imputer),
    ('Classifier', classifier)
])

# Performing cross-validation
scores = cross_val_score(pipeline, X_dis, y_dis, cv=stratified_kfold, scoring='f1_macro', n_jobs=-1)
mean_score = np.mean(scores)
std_score = np.std(scores)

# Printing results
print()
print(classifier.__class__.__name__)
print("Mean F1-macro Score:", mean_score, "std:", std_score)


KNeighborsClassifier
Mean F1-macro Score: 0.8719658963464401 std: 0.0778962832148059


### XGB

#### Performing RandomSearch

In [27]:
import lightgbm as lgb
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import make_scorer, f1_score
import numpy as np

# Defining type of split for CV - this is used for imbalanced datasets to keep equal proportion of class balance between folds
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Defining model and resampling method
model = xgb.XGBClassifier(random_state=42)

params_xgb = {}
params_xgb['classifier__n_estimators'] =  [5, 20, 35, 50, 75, 100, 150, 200, 300, 400, 500, 600, 700, 800, 900, 1000]  #Default = 100. 75 was best during early testing
params_xgb['classifier__max_depth'] = [None, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] #Default = 6
params_xgb['classifier__learning_rate'] = [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1] #Default = 0.3
params_xgb['classifier__min_child_weight'] = [1, 2, 3, 4, 5, 6, 7] #Default = 1
params_xgb['classifier__subsample'] = [0.3, 0.5, 0.6, 0.7, 0.8, 0.9, 1] #Default = 1
params_xgb['classifier__colsample_bytree'] = [0.3, 0.5, 0.7, 0.75, 0.8, 0.85, 0.9, 1] #Default = 1
params_xgb['classifier__n_jobs'] = [-1]

# Defining preprocessing methods for pipeline
scaler = MinMaxScaler()
imputer = SimpleImputer()

# Defining pipeline to avoid data leakage
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),    
    ('classifier', model),  
    ])

############################################################################################

# Initializing RandomizedSearchCV
opt = RandomizedSearchCV(
    pipeline,
    param_distributions=params_xgb,
    n_iter=10000,  # Number of parameter settings that are sampled
    cv=stratified_kfold,  # Number of cross-validation folds
    random_state=42,
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1
)

opt.fit(X_dis_selected, y_dis)

#printing(opt.best_score_)
print(opt.best_score_)
print(opt.best_params_)

Fitting 5 folds for each of 10000 candidates, totalling 50000 fits
0.8952635753908563
{'classifier__subsample': 1, 'classifier__n_jobs': -1, 'classifier__n_estimators': 75, 'classifier__min_child_weight': 1, 'classifier__max_depth': 9, 'classifier__learning_rate': 0.3, 'classifier__colsample_bytree': 1}


#### Validating results

In [33]:
# Defining classifier with best params
classifier = xgb.XGBClassifier(subsample = 1,
n_estimators = 75,
min_child_weight = 1,
max_depth = 9,
learning_rate = 0.3,
colsample_bytree = 1,
random_state=42, n_jobs=-1)

# Defining preprocessing steps
scaler = MinMaxScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('MinMax', scaler),
    ('Simple', imputer),
    ('Classifier', classifier)
])

# Performing cross-validation
scores = cross_val_score(pipeline, X_dis, y_dis, cv=stratified_kfold, scoring='f1_macro', n_jobs=-1)
mean_score = np.mean(scores)
std_score = np.std(scores)

# Printing results
print()
print(classifier.__class__.__name__)
print("Mean F1-macro Score:", mean_score, "std:", std_score)


XGBClassifier
Mean F1-macro Score: 0.8952635753908563 std: 0.06856591410585108


### LGBM

#### Performing RandomSearch

In [28]:
import lightgbm as lgb
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import make_scorer, f1_score
import numpy as np

# Defining type of split for CV - this is used for imbalanced datasets to keep equal proportion of class balance between folds
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Defining model and resampling method
model = lgb.LGBMClassifier(random_state=42)

params_lgbm = {}
params_lgbm['classifier__boosting_type'] = ['gbdt', 'dart']  #Default = gbdt
params_lgbm['classifier__n_estimators'] = [5, 10, 15, 20, 25, 35, 50, 75, 100, 150, 200, 300, 400] #Default = 100. Good performance around 20
params_lgbm['classifier__num_leaves'] = [1, 3, 5, 7, 9, 11, 15] # 20, 25, 31, 40, 50] #Default = 31
params_lgbm['classifier__max_depth'] = [-1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  #Default = -1
params_lgbm['classifier__learning_rate'] = [0.001, 0.05, 0.1, 0.2, 0.3, 0.5, 1]  #Default = 0.1
params_lgbm['classifier__objective'] = ['binary', None] #Default = None. 
params_lgbm['classifier__class_weight'] = [None, 'balanced'] #Default = None
params_lgbm['classifier__colsample_bytree'] = [0.3, 0.5, 0.7, 0.75, 0.8, 0.85, 0.9, 1] #Default = 1

# Defining preprocessing methods for pipeline
scaler = MinMaxScaler()
imputer = SimpleImputer()

# Defining pipeline to avoid data leakage
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),    
    ('classifier', model),  
    ])
############################################################################################

# Initializing RandomizedSearchCV
opt = RandomizedSearchCV(
    pipeline,
    param_distributions=params_lgbm,
    n_iter=5000,  # Number of parameter settings that are sampled
    cv=stratified_kfold,  # Number of cross-validation folds
    random_state=42,
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1
)

opt.fit(X_dis_selected, y_dis)

#printing(opt.best_score_)
print(opt.best_score_)
print(opt.best_params_)

Fitting 5 folds for each of 5000 candidates, totalling 25000 fits


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
3615 fits failed out of a total of 25000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3615 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.10/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/imblearn/pipeline.py", line 326, in fit
    self._final_estimator.fit(Xt, yt, **last_step_params["fit"])
  File "/opt/conda

[LightGBM] [Info] Number of positive: 39, number of negative: 439
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000113 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 336
[LightGBM] [Info] Number of data points in the train set: 478, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.081590 -> initscore=-2.420938
[LightGBM] [Info] Start training from score -2.420938
0.9189097609684165
{'classifier__objective': 'binary', 'classifier__num_leaves': 3, 'classifier__n_estimators': 75, 'classifier__max_depth': 3, 'classifier__learning_rate': 0.3, 'classifier__colsample_bytree': 0.7, 'classifier__class_weight': None, 'classifier__boosting_type': 'dart'}


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_search.py:1051: UserWarning: One or more of the test scores are non-finite: [0.7331352  0.47873578 0.8768325  ... 0.88179247 0.85250686 0.87104133]
  warnings.warn(


#### Validating Results

In [34]:
# Defining classifier with best params
classifier = lgb.LGBMClassifier(objective = 'binary',
num_leaves = 3,
n_estimators = 75,
max_depth = 3,
learning_rate = 0.3,
colsample_bytree = 0.7,
class_weight = None,
boosting_type = 'dart',
random_state=42, n_jobs =-1)

# Defining preprocessing steps
scaler = MinMaxScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('MinMax', scaler),
    ('Simple', imputer),
    ('Classifier', classifier)
])

# Performing cross-validation
scores = cross_val_score(pipeline, X_dis, y_dis, cv=stratified_kfold, scoring='f1_macro', n_jobs=-1)
mean_score = np.mean(scores)
std_score = np.std(scores)

# Printing results
print()
print(classifier.__class__.__name__)
print("Mean F1-macro Score:", mean_score, "std:", std_score)


LGBMClassifier
Mean F1-macro Score: 0.9189097609684165 std: 0.04825543987086979


# Gridsearch results (f1-macro + std)

In [ ]:
# Defining variables for validation segments

# Anxiety present detection
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
selected_features = ['ucla_t6_sum', 'summary_score_kccq_base', 'eq5d5l_index_t6', 'MCS_t6', 'ImplWeight', 'personality_type_D']
X_anx = X_anx[selected_features]

# Distress present detection
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
selected_features = ['ucla_t6_sum', 'eq5d5l_index_t6', 'depression_base_score', 'icdc_t6_sum', 'MCS_t6', 'Return_to_function_t6', 'age', 'anxiety_base_score']
X_dis = X_dis[selected_features]

## Anxiety

### RF

#### Performing GridSearch

In [11]:
# Defining type of split for CV - this is used for imbalanced datasets to keep equal proportion of class balance between folds
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Defining model and resampling method
model = RandomForestClassifier(random_state=42)

# Defining parameter grid
# Number in comments = value chosen in randomsearch
params_rf = {}
params_rf['classifier__n_estimators'] = [60, 70, 80, 90, 100, 110, 120, 130, 140] #100
params_rf['classifier__max_depth'] = [None, 6, 7, 8] #7
params_rf['classifier__min_samples_split'] = [6, 7, 8] #7
params_rf['classifier__min_samples_leaf'] = [2, 3, 4] #3
params_rf['classifier__max_features'] = ['log2', 'sqrt'] #sqrt
params_rf['classifier__class_weight'] = ['balanced', 'balanced_subsample', None] #balanced

# Defining preprocessing methods for pipeline
scaler = RobustScaler()
imputer = SimpleImputer()

# Defining pipeline to avoid data leakage
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),    
    ('classifier', model),  
    ])

############################################################################################

# Initializing RandomizedSearchCV
opt = GridSearchCV(
    pipeline,
    param_grid=params_rf,  # Number of parameter settings that are sampled
    cv=stratified_kfold,  # Number of cross-validation folds
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1
)

opt.fit(X_anx_selected, y_anx)

#printing(opt.best_score_)
print(opt.best_score_)
print(opt.best_params_)

Fitting 5 folds for each of 1944 candidates, totalling 9720 fits
0.9209040685715605
{'classifier__class_weight': 'balanced', 'classifier__max_depth': None, 'classifier__max_features': 'log2', 'classifier__min_samples_leaf': 3, 'classifier__min_samples_split': 7, 'classifier__n_estimators': 90}


#### Validating results

In [23]:
# Defining classifier with best params
classifier = RandomForestClassifier(class_weight = 'balanced',
max_depth = None,
max_features = 'log2',
min_samples_leaf = 3,
min_samples_split = 7,
n_estimators = 90,
random_state=42)

# Defining preprocessing steps
scaler = RobustScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),
    ('Classifier', classifier)
])

# Performing cross-validation
scores = cross_val_score(pipeline, X_anx, y_anx, cv=stratified_kfold, scoring='f1_macro', n_jobs=-1)
mean_score = np.mean(scores)
std_score = np.std(scores)

# Printing results
print()
print(classifier.__class__.__name__)
print("Mean F1-macro Score:", mean_score, "std:", std_score)


RandomForestClassifier
Mean F1-macro Score: 0.9209040685715605 std: 0.05209723108198606


### SVM

#### Performing GridSearch

In [79]:
import lightgbm as lgb
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
import numpy as np
from sklearn.svm import SVC

# Defining type of split for CV - this is used for imbalanced datasets to keep equal proportion of class balance between folds
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Defining model and resampling method
model = SVC(random_state=42)

# Defining parameter grid
params_svc = {
    'classifier__C': [0.25, 0.5, 0.75, 0.875, 1.0, 1.125, 1.25, 1.5, 1.75],                      #1.0
    'classifier__kernel': ['rbf'],                                  #rbf
    'classifier__degree': [12, 13, 14, 15, 16, 17, 18],             #15
    'classifier__gamma': ['scale', 'auto'],                         #auto
    'classifier__coef0': [0.05, 0.1, 0.2, 0.3, 0.4],                                                   #0.1
    'classifier__shrinking': [True],                                                      #True
    'classifier__probability': [False],                                                    #False
    'classifier__class_weight': ['balanced', None],                                              #None
    'classifier__max_iter': [7000, 8000, 9000]   #8000
}

# Defining preprocessing methods for pipeline
scaler = RobustScaler()
imputer = SimpleImputer()

pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),    
    ('classifier', model),  
    ])

############################################################################################

# Initializing RandomizedSearchCV
opt = GridSearchCV(
    pipeline,
    param_grid=params_svc,  # Number of parameter settings that are sampled
    cv=stratified_kfold,  # Number of cross-validation folds
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1
)

opt.fit(X_anx_selected, y_anx)

#printing(opt.best_score_)
print(opt.best_score_)
print(opt.best_params_)

Fitting 5 folds for each of 12960 candidates, totalling 64800 fits
0.8889376147546265
{'classifier__C': 50, 'classifier__class_weight': None, 'classifier__coef0': 0.3, 'classifier__degree': 5, 'classifier__gamma': 'auto', 'classifier__kernel': 'sigmoid', 'classifier__max_iter': 5000, 'classifier__probability': True, 'classifier__shrinking': True}


#### Validating results

In [25]:
# Defining classifier with best params
classifier = SVC(C = 1.0,
class_weight = None,
coef0 = 0.05,
degree = 12,
gamma = 'auto',
kernel = 'rbf',
max_iter = 7000,
probability = False,
shrinking = True,
random_state=42)

# Defining preprocessing steps
scaler = RobustScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),
    ('Classifier', classifier)
])

# Performing cross-validation
scores = cross_val_score(pipeline, X_anx, y_anx, cv=stratified_kfold, scoring='f1_macro', n_jobs=-1)
mean_score = np.mean(scores)
std_score = np.std(scores)

# Printing results
print()
print(classifier.__class__.__name__)
print("Mean F1-macro Score:", mean_score, "std:", std_score)


SVC
Mean F1-macro Score: 0.9126825719632198 std: 0.08203534839628797


### KNN

#### Performing GridSearch

In [13]:
import lightgbm as lgb
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import make_scorer, f1_score
import numpy as np

# Defining type of split for CV - this is used for imbalanced datasets to keep equal proportion of class balance between folds
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Defining model and resampling method
model = KNeighborsClassifier()

params_knn = {} 
params_knn['classifier__n_neighbors'] = list(range(1, 41, 1)) #Default = 5
params_knn['classifier__weights'] = ['uniform', 'distance'] #Default = uniform
params_knn['classifier__algorithm'] = ['auto', 'ball_tree', 'kd_tree', 'brute'] #Default = Auto
params_knn['classifier__leaf_size'] = [10, 20, 30, 40, 50] # Default = 30. Only for Balltree and KDTree
params_knn['classifier__p'] = [1, 2] #Default = 2

# Defining preprocessing methods for pipeline
scaler = RobustScaler()
imputer = SimpleImputer()


# Defining pipeline to avoid data leakage
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),  
    ('classifier', model),  
    ])

############################################################################################

# Initializing RandomizedSearchCV
opt = GridSearchCV(
    pipeline,
    param_grid=params_knn,  # Number of parameter settings that are sampled
    cv=stratified_kfold,  # Number of cross-validation folds
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1
)

opt.fit(X_anx_selected, y_anx)

#printing(opt.best_score_)
print(opt.best_score_)
print(opt.best_params_)

Fitting 5 folds for each of 3200 candidates, totalling 16000 fits
0.912602819606272
{'classifier__algorithm': 'auto', 'classifier__leaf_size': 10, 'classifier__n_neighbors': 5, 'classifier__p': 1, 'classifier__weights': 'uniform'}


#### Validating results

In [3]:
# Defining classifier with best params
classifier = KNeighborsClassifier(algorithm = 'auto',
leaf_size = 10,
n_neighbors = 5,
p = 1,
weights = 'uniform',
n_jobs=-1)

# Defining preprocessing steps
scaler = RobustScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),
    ('Classifier', classifier)
])

# Performing cross-validation
scores = cross_val_score(pipeline, X_anx, y_anx, cv=stratified_kfold, scoring='f1_macro', n_jobs=-1)
mean_score = np.mean(scores)
std_score = np.std(scores)

# Printing results
print()
print(classifier.__class__.__name__)
print("Mean F1-macro Score:", mean_score, "std:", std_score)


KNeighborsClassifier
Mean F1-macro Score: 0.912602819606272 std: 0.07830052829708159


### XGB

#### Performing GridSearch

In [14]:
import lightgbm as lgb
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import make_scorer, f1_score
import numpy as np

# Defining type of split for CV - this is used for imbalanced datasets to keep equal proportion of class balance between folds
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Defining model and resampling method
model = xgb.XGBClassifier(random_state=42)

params_xgb = {}
params_xgb['classifier__n_estimators'] =  [100, 112, 125, 137, 150, 162, 175, 187, 200]  # 150
params_xgb['classifier__max_depth'] = [3, 4, 5] # 3
params_xgb['classifier__learning_rate'] = [0.4, 0.45, 0.5, 0.55, 0.6] # 0.5
params_xgb['classifier__min_child_weight'] = [1] #  1
params_xgb['classifier__subsample'] = [0.8, 0.9, 1] #  1
params_xgb['classifier__colsample_bytree'] = [0.9, 0,95, 1] #Default = 1

# Defining preprocessing methods for pipeline
scaler = RobustScaler()
imputer = SimpleImputer()

# Defining pipeline to avoid data leakage
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),    
    ('classifier', model),  
    ])

############################################################################################

# Initializing RandomizedSearchCV
opt = GridSearchCV(
    pipeline,
    param_grid=params_xgb,  # Number of parameter settings that are sampled
    cv=stratified_kfold,  # Number of cross-validation folds
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1
)

opt.fit(X_anx_selected, y_anx)

#printing(opt.best_score_)
print(opt.best_score_)
print(opt.best_params_)

Fitting 5 folds for each of 1620 candidates, totalling 8100 fits
0.9209335388734046
{'classifier__colsample_bytree': 0.9, 'classifier__learning_rate': 0.4, 'classifier__max_depth': 3, 'classifier__min_child_weight': 1, 'classifier__n_estimators': 162, 'classifier__subsample': 0.9}


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
2025 fits failed out of a total of 8100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2025 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.10/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/imblearn/pipeline.py", line 326, in fit
    self._final_estimator.fit(Xt, yt, **last_step_params["fit"])
  File "/opt/conda/

#### Validating results

In [27]:
# Defining classifier with best params
classifier = xgb.XGBClassifier(colsample_bytree = 0.9,
learning_rate = 0.4,
max_depth = 3,
min_child_weight = 1,
n_estimators = 162,
subsample = 0.9,
random_state=42, n_jobs=-1)

# Defining preprocessing steps
scaler = RobustScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),
    ('Classifier', classifier)
])

# Performing cross-validation
scores = cross_val_score(pipeline, X_anx, y_anx, cv=stratified_kfold, scoring='f1_macro', n_jobs=-1)
mean_score = np.mean(scores)
std_score = np.std(scores)

# Printing results
print()
print(classifier.__class__.__name__)
print("Mean F1-macro Score:", mean_score, "std:", std_score)


XGBClassifier
Mean F1-macro Score: 0.9209335388734046 std: 0.08810354178811651


### LGBM

#### Performing GridSearch

In [83]:
import lightgbm as lgb
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import make_scorer, f1_score
import numpy as np

# Defining type of split for CV - this is used for imbalanced datasets to keep equal proportion of class balance between folds
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Defining model and resampling method
model = lgb.LGBMClassifier(random_state=42, n_jobs =-1)

params_lgbm = {}
params_lgbm['classifier__boosting_type'] = ['gbdt', 'dart']  # gbdt
params_lgbm['classifier__n_estimators'] = [10, 15, 20, 25, 30] # 20
params_lgbm['classifier__num_leaves'] = [10, 13, 15, 17, 20] #np.arange(0.10, 0.21, 0.01).tolist() # 15 
params_lgbm['classifier__max_depth'] = [5, 6, 7]  # 6
params_lgbm['classifier__learning_rate'] = [0.05, 0.1, 0.15]  # 0.1
params_lgbm['classifier__objective'] = ['binary', None] # None
params_lgbm['classifier__class_weight'] = [None] # None
params_lgbm['classifier__colsample_bytree'] = [0.7, 0.75, 0.8] #np.arange(0.7, 0.81, 0.01).tolist() #0.75

# Defining preprocessing methods for pipeline
scaler = RobustScaler()
imputer = SimpleImputer()

# Defining pipeline to avoid data leakage
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),    
    ('classifier', model),  
    ])

############################################################################################

# Initializing RandomizedSearchCV
opt = GridSearchCV(
    pipeline,
    param_grid=params_lgbm,  # Number of parameter settings that are sampled
    cv=stratified_kfold,  # Number of cross-validation folds
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1
)

opt.fit(X_anx_selected, y_anx)

#printing(opt.best_score_)
print(opt.best_score_)
print(opt.best_params_)

Fitting 5 folds for each of 2700 candidates, totalling 13500 fits
[LightGBM] [Info] Number of positive: 35, number of negative: 443
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000088 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 309
[LightGBM] [Info] Number of data points in the train set: 478, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.073222 -> initscore=-2.538222
[LightGBM] [Info] Start training from score -2.538222
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

#### Validating results

In [28]:
# Defining classifier with best params
classifier = lgb.LGBMClassifier(boosting_type = 'gbdt',
class_weight = None,
colsample_bytree = 0.75,
learning_rate = 0.1,
max_depth = 5,
n_estimators = 20,
num_leaves = 10,
objective = 'binary',
random_state=42, n_jobs =-1)

# Defining preprocessing steps
scaler = RobustScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),
    ('Classifier', classifier)
])

# Performing cross-validation
scores = cross_val_score(pipeline, X_anx, y_anx, cv=stratified_kfold, scoring='f1_macro', n_jobs=-1)
mean_score = np.mean(scores)
std_score = np.std(scores)

# Printing results
print()
print(classifier.__class__.__name__)
print("Mean F1-macro Score:", mean_score, "std:", std_score)


LGBMClassifier
Mean F1-macro Score: 0.910054117835578 std: 0.07139540921071352


## Distressed

### RF

#### Performing GridSearch

In [78]:
import lightgbm as lgb
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
import numpy as np

# Defining type of split for CV - this is used for imbalanced datasets to keep equal proportion of class balance between folds
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Defining model and resampling method
model = RandomForestClassifier(random_state=42)

# Defining parameter grid
params_rf = {}
params_rf['classifier__n_estimators'] = np.arange(40, 67, 2).tolist() #50
params_rf['classifier__max_depth'] = [6, 7, 8] #7
params_rf['classifier__min_samples_split'] = [2, 3, 4] #3
params_rf['classifier__min_samples_leaf'] = [1, 2] #1
params_rf['classifier__max_features'] = ['log2', 'sqrt'] #sqrt
params_rf['classifier__class_weight'] = ['balanced', 'balanced_subsample', None] #None

# Defining preprocessing methods for pipeline
scaler = MinMaxScaler()
imputer = SimpleImputer()

# Defining pipeline to avoid data leakage
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),    
    ('classifier', model),  
    ])

############################################################################################

# Initializing RandomizedSearchCV
opt = GridSearchCV(
    pipeline,
    param_grid=params_rf,  # Number of parameter settings that are sampled
    cv=stratified_kfold,  # Number of cross-validation fold
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1
)

opt.fit(X_dis_selected, y_dis)

#printing(opt.best_score_)
print(opt.best_score_)
print(opt.best_params_)

Fitting 5 folds for each of 1512 candidates, totalling 7560 fits
0.9154748466102941
{'classifier__class_weight': None, 'classifier__max_depth': 6, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 66}


#### Validating results

In [35]:
# Defining classifier with best params
classifier = RandomForestClassifier(class_weight = None,
max_depth = 6,
max_features = 'sqrt',
min_samples_leaf = 1,
min_samples_split = 2,
n_estimators = 66,
random_state=42)

# Defining preprocessing steps
scaler = MinMaxScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('MinMax', scaler),
    ('Simple', imputer),
    ('Classifier', classifier)
])

# Performing cross-validation
scores = cross_val_score(pipeline, X_dis, y_dis, cv=stratified_kfold, scoring='f1_macro', n_jobs=-1)
mean_score = np.mean(scores)
std_score = np.std(scores)

# Printing results
print()
print(classifier.__class__.__name__)
print("Mean F1-macro Score:", mean_score, "std:", std_score)


RandomForestClassifier
Mean F1-macro Score: 0.9154748466102941 std: 0.08290499798612358


### SVM

#### Performing GridSearch

In [79]:
import lightgbm as lgb
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
import numpy as np
from sklearn.svm import SVC

# Defining type of split for CV - this is used for imbalanced datasets to keep equal proportion of class balance between folds
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Defining model and resampling method
model = SVC(random_state=42)

# Defining parameter grid
params_svc = {}
params_svc['classifier__C'] = [25, 50, 75, 100, 125, 150, 250, 500, 750]  #100
params_svc['classifier__kernel'] = ['rbf', 'sigmoid'] #sigmoid
params_svc['classifier__degree'] = [5, 6, 7] #6
params_svc['classifier__gamma'] = ['scale', 'auto'] #auto
params_svc['classifier__coef0'] = [0.3, 0.4, 0.5, 0.6, 0.7]  #0.5 
params_svc['classifier__shrinking'] = [True, False] #False
params_svc['classifier__probability'] = [True, False] #False
params_svc['classifier__class_weight'] = ['balanced', None] #None
params_svc['classifier__max_iter'] = [5000, 6000, 7000] #6000

# Defining preprocessing methods for pipeline
scaler = MinMaxScaler()
imputer = SimpleImputer()

pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),    
    ('classifier', model),  
    ])


############################################################################################

# Initializing RandomizedSearchCV
opt = GridSearchCV(
    pipeline,
    param_grid=params_svc,  # Number of parameter settings that are sampled
    cv=stratified_kfold,  # Number of cross-validation folds
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1
)

opt.fit(X_dis_selected, y_dis)

#printing(opt.best_score_)
print(opt.best_score_)
print(opt.best_params_)

Fitting 5 folds for each of 12960 candidates, totalling 64800 fits
0.8889376147546265
{'classifier__C': 50, 'classifier__class_weight': None, 'classifier__coef0': 0.3, 'classifier__degree': 5, 'classifier__gamma': 'auto', 'classifier__kernel': 'sigmoid', 'classifier__max_iter': 5000, 'classifier__probability': True, 'classifier__shrinking': True}


#### Validating results

In [36]:
# Defining classifier with best params
classifier = SVC(C = 50,
class_weight = None,
coef0 = 0.3,
degree = 5,
gamma = 'auto',
kernel = 'sigmoid',
max_iter = 5000,
probability = True,
shrinking = True,
random_state=42)

# Defining preprocessing steps
scaler = MinMaxScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('MinMax', scaler),
    ('Simple', imputer),
    ('Classifier', classifier)
])

# Performing cross-validation
scores = cross_val_score(pipeline, X_dis, y_dis, cv=stratified_kfold, scoring='f1_macro', n_jobs=-1)
mean_score = np.mean(scores)
std_score = np.std(scores)

# Printing results
print()
print(classifier.__class__.__name__)
print("Mean F1-macro Score:", mean_score, "std:", std_score)


SVC
Mean F1-macro Score: 0.8889376147546265 std: 0.11453618279448234


### KNN

#### Performing GridSearch

In [80]:
import lightgbm as lgb
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import make_scorer, f1_score
import numpy as np

# Defining type of split for CV - this is used for imbalanced datasets to keep equal proportion of class balance between folds
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Defining model and resampling method
model = KNeighborsClassifier()

params_knn = {} 
params_knn['classifier__n_neighbors'] = np.arange(1, 11, 1).tolist() # 5
params_knn['classifier__weights'] = ['uniform', 'distance'] # uniform
params_knn['classifier__algorithm'] = ['auto', 'ball_tree', 'kd_tree', 'brute'] # Auto
params_knn['classifier__leaf_size'] = [5, 8, 10, 12, 15, 20, 30] # 10
params_knn['classifier__p'] = [1, 2] #Default = 2

# Defining preprocessing methods for pipeline
scaler = MinMaxScaler()
imputer = SimpleImputer()

# Defining pipeline to avoid data leakage
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),    
    ('classifier', model),  
    ])

############################################################################################

# Initializing RandomizedSearchCV
opt = GridSearchCV(
    pipeline,
    param_grid=params_knn,  # Number of parameter settings that are sampled
    cv=stratified_kfold,  # Number of cross-validation folds
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1
)

opt.fit(X_dis_selected, y_dis)

#printing(opt.best_score_)
print(opt.best_score_)
print(opt.best_params_)

Fitting 5 folds for each of 1120 candidates, totalling 5600 fits
0.8929530181562623
{'classifier__algorithm': 'auto', 'classifier__leaf_size': 5, 'classifier__n_neighbors': 6, 'classifier__p': 2, 'classifier__weights': 'distance'}


#### Validating results

In [37]:
# Defining classifier with best params
classifier = KNeighborsClassifier(algorithm = 'auto',
leaf_size = 5,
n_neighbors = 6,
p = 2,
weights = 'distance',
n_jobs=-1)

# Defining preprocessing steps
scaler = MinMaxScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('MinMax', scaler),
    ('Simple', imputer),
    ('Classifier', classifier)
])

# Performing cross-validation
scores = cross_val_score(pipeline, X_dis, y_dis, cv=stratified_kfold, scoring='f1_macro', n_jobs=-1)
mean_score = np.mean(scores)
std_score = np.std(scores)

# Printing results
print()
print(classifier.__class__.__name__)
print("Mean F1-macro Score:", mean_score, "std:", std_score)


KNeighborsClassifier
Mean F1-macro Score: 0.8929530181562623 std: 0.07793678442493912


### XGB

#### Performing GridSearch

In [81]:
import lightgbm as lgb
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import make_scorer, f1_score
import numpy as np

# Defining type of split for CV - this is used for imbalanced datasets to keep equal proportion of class balance between folds
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Defining model and resampling method
model = xgb.XGBClassifier(random_state=42)

params_xgb = {}
params_xgb['classifier__n_estimators'] =  [50, 63, 75, 87, 100]  #75
params_xgb['classifier__max_depth'] = [8, 9, 10] # 9
params_xgb['classifier__learning_rate'] = np.arange(0.2, 0.41, 0.01).tolist() # 0.3
params_xgb['classifier__min_child_weight'] = [1, 2] # 1
params_xgb['classifier__subsample'] = [0.9, 1] # = 1
params_xgb['classifier__colsample_bytree'] = [0.9, 1] #Default = 1

# Defining preprocessing methods for pipeline
scaler = MinMaxScaler()
imputer = SimpleImputer()

# Defining pipeline to avoid data leakage
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),    
    ('classifier', model),  
    ])

############################################################################################

# Initializing RandomizedSearchCV
opt = GridSearchCV(
    pipeline,
    param_grid=params_xgb,  # Number of parameter settings that are sampled
    cv=stratified_kfold,  # Number of cross-validation folds
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1
)

opt.fit(X_dis_selected, y_dis)

#printing(opt.best_score_)
print(opt.best_score_)
print(opt.best_params_)

Fitting 5 folds for each of 2520 candidates, totalling 12600 fits
0.8952635753908563
{'classifier__colsample_bytree': 1, 'classifier__learning_rate': 0.3000000000000001, 'classifier__max_depth': 8, 'classifier__min_child_weight': 1, 'classifier__n_estimators': 63, 'classifier__subsample': 1}


#### Validating results

In [38]:
# Defining classifier with best params
classifier = xgb.XGBClassifier(colsample_bytree = 1,
learning_rate = 0.3,
max_depth = 8,
min_child_weight = 1,
n_estimators = 63,
subsample = 1,
random_state=42, n_jobs=-1)

# Defining preprocessing steps
scaler = MinMaxScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('MinMax', scaler),
    ('Simple', imputer),
    ('Classifier', classifier)
])

# Performing cross-validation
scores = cross_val_score(pipeline, X_dis, y_dis, cv=stratified_kfold, scoring='f1_macro', n_jobs=-1)
mean_score = np.mean(scores)
std_score = np.std(scores)

# Printing results
print()
print(classifier.__class__.__name__)
print("Mean F1-macro Score:", mean_score, "std:", std_score)


XGBClassifier
Mean F1-macro Score: 0.8952635753908563 std: 0.06856591410585108


### LGBM

#### Performing GridSearch

In [82]:
import lightgbm as lgb
from imblearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import make_scorer, f1_score
import numpy as np

# Defining type of split for CV - this is used for imbalanced datasets to keep equal proportion of class balance between folds
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Defining model and resampling method
model = lgb.LGBMClassifier(random_state=42)

params_lgbm = {}
params_lgbm['classifier__boosting_type'] = ['gbdt', 'dart']  # dart
params_lgbm['classifier__n_estimators'] = [50, 63, 75, 87, 100] # 75
params_lgbm['classifier__num_leaves'] = [2, 3, 4] # 3
params_lgbm['classifier__max_depth'] = [2, 3, 4,]  # 3
params_lgbm['classifier__learning_rate'] = [0.2, 0.25, 0.3, 0.35, 0.4]  # 0.3 
params_lgbm['classifier__objective'] = ['binary'] # binary. 
params_lgbm['classifier__class_weight'] = [None] # None
params_lgbm['classifier__colsample_bytree'] = [0.6, 0.65, 0.7, 0.75, 0.8] # 0.7

# Defining preprocessing methods for pipeline
scaler = MinMaxScaler()
imputer = SimpleImputer()

# Defining pipeline to avoid data leakage
pipeline = Pipeline(steps=[
    ('Robust', scaler),
    ('Simple', imputer),    
    ('classifier', model),  
    ])
############################################################################################

# Initializing RandomizedSearchCV
opt = GridSearchCV(
    pipeline,
    param_grid=params_lgbm,  # Number of parameter settings that are sampled
    cv=stratified_kfold,  # Number of cross-validation folds
    scoring='f1_macro',
    verbose=1,
    n_jobs=-1
)

opt.fit(X_dis_selected, y_dis)

#printing(opt.best_score_)
print(opt.best_score_)
print(opt.best_params_)

Fitting 5 folds for each of 2250 candidates, totalling 11250 fits
[LightGBM] [Info] Number of positive: 39, number of negative: 439
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000114 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 336
[LightGBM] [Info] Number of data points in the train set: 478, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.081590 -> initscore=-2.420938
[LightGBM] [Info] Start training from score -2.420938
0.9270874323416697
{'classifier__boosting_type': 'dart', 'classifier__class_weight': None, 'classifier__colsample_bytree': 0.7, 'classifier__learning_rate': 0.2, 'classifier__max_depth': 2, 'classifier__n_estimators': 87, 'classifier__num_leaves': 3, 'classifier__objective': 'binary'}


#### Validating results

In [39]:
# Defining classifier with best params
classifier = lgb.LGBMClassifier(boosting_type = 'dart',
class_weight = None,
colsample_bytree = 0.7,
learning_rate = 0.2,
max_depth = 2,
n_estimators = 87,
num_leaves = 3,
objective = 'binary',
random_state=42, n_jobs =-1)

# Defining preprocessing steps
scaler = MinMaxScaler()
imputer = SimpleImputer()

# Defining pipeline
pipeline = Pipeline(steps=[
    ('MinMax', scaler),
    ('Simple', imputer),
    ('Classifier', classifier)
])

# Performing cross-validation
scores = cross_val_score(pipeline, X_dis, y_dis, cv=stratified_kfold, scoring='f1_macro', n_jobs=-1)
mean_score = np.mean(scores)
std_score = np.std(scores)

# Printing results
print()
print(classifier.__class__.__name__)
print("Mean F1-macro Score:", mean_score, "std:", std_score)


LGBMClassifier
Mean F1-macro Score: 0.9270874323416697 std: 0.05159671162606029
